In [1]:
import numpy as np

# Inverse kinematic

## Symbol calculations

In [2]:
import sympy
sympy.init_printing()

In [3]:
Rx, Ry = sympy.symbols("R_x, R_y")
rx, ry = sympy.symbols("r_x, r_y")
vx, vy, w = sympy.symbols("v_x, v_y, \omega")

In [4]:
V = sympy.Matrix([vx, vy, 0]) + sympy.Matrix([0, 0, w]).cross(sympy.Matrix([Rx, Ry, 0]))
V_m = sympy.Matrix(V[0:2]).dot(sympy.Matrix([rx, ry]))
V_m = sympy.expand(V_m)

In [5]:
V_m

## Numerical calculations

In [26]:
Rx1 = 175.27
Ry1 = 120.27 / 2
a = np.pi / 4
rwheel = 28.8

rx1 = np.cos(a)
ry1 = np.sin(a)
angles = [np.pi / 4,  65 / 180 * np.pi, ]
Rxs = [Rx1, Rx1, -Rx1, -Rx1]
Rys = [Ry1, -Ry1, Ry1, -Ry1]
rxs = [rx1, -rx1, rx1, -rx1]
rys = [-ry1, -ry1, ry1, ry1]

In [27]:
forward_matrix = np.zeros((4, 3), dtype=np.float32)
for i in range(4):
    forward_matrix[i] = np.array([rxs[i], rys[i], Rxs[i] * rys[i] - Rys[i] * rxs[i]])
forward_matrix = forward_matrix / rwheel
forward_matrix[:, 0:2] = forward_matrix[:, 0:2] * 1000 

In [28]:
forward_matrix * np.array([1, -1, 1, -1])[:, np.newaxis]

array([[ 24.55231857, -24.55231857,  -5.77973843],
       [ 24.55231857,  24.55231857,   5.77973843],
       [ 24.55231857,  24.55231857,  -5.77973843],
       [ 24.55231857, -24.55231857,   5.77973843]])

## Different angles for big robot center in geom center

<img src="files/photo_big_robot_2.jpg">

In [34]:
Rx1 = 175.82
Rx2 = 172.15
Ry1 = 60.82
Ry2 = 62.84

Rxs = [-Rx1, -Rx2, Rx2, Rx1]
Rys = [Ry1, -Ry2, -Ry2, Ry1]
alphas = (np.array([180 - 45, 180 + 65, -65, 45]) - 90) * np.pi / 180
rxs = np.cos(alphas)
rys = np.sin(alphas)

In [35]:
forward_matrix = np.zeros((4, 3), dtype=np.float32)
for i in range(4):
    forward_matrix[i] = np.array([rxs[i], rys[i], Rxs[i] * rys[i] - Rys[i] * rxs[i]])
forward_matrix = forward_matrix / rwheel
forward_matrix[:, 0:2] = forward_matrix[:, 0:2] * 1000 

In [36]:
forward_matrix * np.array([-1, 1, 1, -1])[:, np.newaxis]

array([[-24.55231857, -24.55231857,   5.8100605 ],
       [-31.46902084,  14.67424583,  -4.50368452],
       [-31.46902084, -14.67424583,  -4.50368452],
       [-24.55231857,  24.55231857,   5.8100605 ]])

In [37]:
np.linalg.lstsq(forward_matrix, np.eye(4), rcond=None)

(array([[ 0.00767466, -0.00990083, -0.00990083,  0.00767466],
        [ 0.01500478,  0.00896795, -0.00896795, -0.01500478],
        [-0.05362585, -0.04183921, -0.04183921, -0.05362585]]),
 array([0.13159783, 0.36840217, 0.36840217, 0.13159783]),
 3,
 array([56.44672134, 40.45120117, 10.3960976 ]))

# Forward kinematic

In [13]:
np.linalg.lstsq(forward_matrix, np.eye(4), rcond=None)

(array([[ 0.00799443, -0.01031336,  0.00799443, -0.01031336],
        [-0.01562998, -0.00934161,  0.01562998,  0.00934161],
        [-0.05586025, -0.04358251, -0.05586025, -0.04358251]]),
 array([0.13159782, 0.36840218, 0.13159782, 0.36840218]),
 3,
 array([54.18885364, 38.83315378,  9.98025398]))

# Calibration

In [37]:
y_exp = 1780
y_true = 1780 - 85 
x_exp = 1780
x_true = 1780 - 70
a_exp = 2 * np.pi
a_true = 2 * np.pi - 0.13

In [38]:
k = np.array([x_exp / x_true, y_exp / y_true, a_exp / a_true])

In [39]:
forward_matrix_1 = forward_matrix * k[np.newaxis, :]

In [40]:
forward_matrix_1 * np.array([1, -1, 1, -1])[:, np.newaxis]

array([[ 24.53508975, -24.75221444,  -5.66577438],
       [ 24.53508975,  24.75221444,   5.66577438],
       [ 24.53508975,  24.75221444,  -5.66577438],
       [ 24.53508975, -24.75221444,   5.66577438]])

In [41]:
np.linalg.lstsq(forward_matrix_1, np.eye(4))

/home/mikhail/.local/lib/python3.5/site-packages/ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


(array([[ 0.01018949, -0.01018949,  0.01018949, -0.01018949],
        [-0.01010011, -0.01010011,  0.01010011,  0.01010011],
        [-0.0441246 , -0.0441246 , -0.0441246 , -0.0441246 ]]),
 array([0.25, 0.25, 0.25, 0.25]),
 3,
 array([49.50442887, 49.0701795 , 11.33154876]))